In [1]:
import os
import gc
import time
import random

import numpy as np
import pandas as pd

import pyarrow.parquet as pq

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import matthews_corrcoef

from tqdm import tqdm

tqdm.pandas()

In [2]:
N_SPLITS = 5
sample_size = 800000
n_dim = 160

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
        score = matthews_corrcoef(y_true, y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {"threshold": best_threshold, "mcc": best_score}
    return search_result

In [4]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, n_attention, n_middle, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.n_middle = n_middle
        self.n_attention = n_attention
        self.features_dim = 0
        
        self.lin1 = nn.Linear(feature_dim, n_middle)
        self.lin2 = nn.Linear(n_middle, n_attention)
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = self.lin1(x)

        eij = torch.tanh(eij)
        
        eij = self.lin2(eij)

        a = torch.exp(eij).reshape(-1, self.n_attention, step_dim)
        
        if mask is not None:
            a = a * mask
        a = a / torch.sum(a, 2, keepdim=True) + 1e-10
        
        weighted_input = torch.bmm(a, x)
        
        return weighted_input, a

## Data

In [3]:
df_train = pd.read_csv("../../script/input/metadata_train.csv")
df_train = df_train.set_index(["id_measurement", "phase"])
df_train.head()

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1

In [5]:
df_train.loc[0].loc[0]

signal_id    0
target       0
Name: 0, dtype: int64

In [7]:
df_partial = df_train[["signal_id"]]
df_partial.head()

signal_id
id_measurement phase           
0              0              0
               1              1
               2              2
1              0              3
               1              4

In [6]:
max_num = 127
min_num = -128

In [7]:
def min_max_transf(ts, min_data, max_data, range_needed=(-1,1)):
    if min_data < 0:
        ts_std = (ts + abs(min_data)) / (max_data + abs(min_data))
    else:
        ts_std = (ts - min_data) / (max_data - min_data)
    if range_needed[0] < 0:    
        return ts_std * (range_needed[1] + abs(range_needed[0])) + range_needed[0]
    else:
        return ts_std * (range_needed[1] - range_needed[0]) + range_needed[0]

In [8]:
def transform_ts(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        # I think that the percentiles are very important, it is like a distribuiton analysis from eath chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100]) 
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        # now, we just add all the features to new_ts and convert it to np.array
        new_ts.append(np.concatenate([np.asarray([mean, std, std_top, std_bot, max_range]),percentil_calc, relative_percentile]))
    return np.asarray(new_ts)

In [9]:
def prep_data(start, end):
    # load a piece of data from file
    praq_train = pq.read_pandas('../input/train.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    X = []
    y = []
    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    bucket_size = int(sample_size / n_dim)
    idx = np.array([i for i in range(n_dim) for _ in range(bucket_size)])
    praq_train["dummy"] = idx
    for id_measurement in tqdm(df_train.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df_train.loc[id_measurement].loc[phase]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                y.append(target)
            # extract and transform data into sets of features
            trns = transform_ts(praq_train[str(signal_id)], n_dim=n_dim)
            X_signal.append(trns)
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [10]:
X = []
y = []
def load_all():
    total_size = len(df_train)
    for ini, end in [(0, int(total_size/2)), (int(total_size/2), total_size)]:
        X_temp, y_temp = prep_data(ini, end)
        X.append(X_temp)
        y.append(y_temp)
load_all()
X = np.concatenate(X)
y = np.concatenate(y)

100%|██████████| 1452/1452 [07:33<00:00,  3.22it/s]


In [11]:
print(X.shape, y.shape)
np.save("X.npy", X)
np.save("y.npy", y)

(2904, 160, 57) (2904,)


In [12]:
gc.collect()

140

## Model

In [13]:
class LSTMNet(nn.Module):
    def __init__(self, hidden_size, linear_size, input_shape, n_attention):
        super(LSTMNet, self).__init__()
        self.maxlen = input_shape[1]
        self.input_dim = input_shape[2]
        self.n_attention = n_attention
        self.hidden_size = hidden_size
        self.lstm1 = nn.LSTM(
            self.input_dim, 
            hidden_size, 
            bidirectional=True,
            batch_first=True)
        self.lstm2 = nn.LSTM(
            hidden_size * 2,
            int(hidden_size / 2),
            bidirectional=True,
            batch_first=True)
        self.attn = Attention(int(hidden_size / 2) * 2, self.maxlen, n_attention, n_attention)
        self.drop = nn.Dropout(0.3)
        self.lin1 = nn.Linear(int(hidden_size / 2) * 2 * n_attention, linear_size)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(linear_size, 1)
        
    def forward(self, x):
        h_lstm1, _ = self.lstm1(x)
        h_lstm2, _ = self.lstm2(h_lstm1)
        attn, a = self.attn(h_lstm2)
        attn = attn.reshape(
            -1, int(self.hidden_size / 2) * 2 * self.n_attention)
        #drop =self.drop(attn)
        lin = self.relu(self.lin1(attn))
        out = self.lin2(lin)
        return out, a

In [14]:
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [15]:
def train_model(model, X_train, y_train, X_val, y_val, filename, anneal=True, epochs=50, seed=1029):
    optimizer = optim.Adam(model.parameters())
    if anneal:
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    train = torch.utils.data.TensorDataset(X_train, y_train)
    valid = torch.utils.data.TensorDataset(X_val, y_val)
    
    seed_torch(seed)
    train_batch_size = 128
    val_batch_size = 512
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=train_batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=val_batch_size, shuffle=False)
    
    loss_fn = torch.nn.BCEWithLogitsLoss(reduction="mean").cuda()
    best_score = -np.inf
    for epoch in range(epochs):
        t0 = time.time()
        model.train()
        avg_loss = 0.
        
        for (x_batch, y_batch) in tqdm(train_loader, disable=True):
            y_pred, a = model(x_batch)
            
            B = y_pred.size(0)
            AAT = torch.bmm(a, a.transpose(1, 2))
            I = torch.eye(model.n_attention).unsqueeze(0).repeat(B, 1, 1).cuda()
            penalization_term = torch.norm(AAT - I) / B

            loss = loss_fn(y_pred, y_batch) + 0 * penalization_term
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
            
        model.eval()

        valid_preds = np.zeros((X_val.size(0)))
        avg_val_loss = 0.
        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_pred, _ = model(x_batch)
            y_pred = y_pred.detach()
            avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
            valid_preds[i * val_batch_size:(i + 1) * val_batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
            
        search_result = threshold_search(y_val.cpu().numpy(), valid_preds)
        val_mcc, val_threshold = search_result["mcc"], search_result["threshold"]
        elapsed_time = time.time() - t0
        print(
            f"Epoch {epoch+1}/{epochs} loss={avg_loss:.4f} val_loss={avg_val_loss:.4f} val_mcc={val_mcc:.4f} best_t={val_threshold:.2f} time={elapsed_time:.2f}s")
        if anneal:
            scheduler.step()
            
        if val_mcc > best_score:
            torch.save(model.state_dict(), filename)
            print(f"Save Model on epoch {epoch + 1}")
            best_score = val_mcc
            
    avg_val_loss = 0.
    valid_preds = np.zeros(X_val.size(0))
    model.load_state_dict(torch.load(filename))
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred, _ = model(x_batch)
        y_pred = y_pred.detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        valid_preds[i * val_batch_size:(i + 1) * val_batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
    print("Validation loss: ", avg_val_loss)
    return valid_preds

In [16]:
fold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019)
train_preds = np.zeros((X.shape[0],))

for i, (trn_index, val_index) in enumerate(fold.split(X, y)):
    X_train = torch.tensor(X[trn_index], dtype=torch.float32).cuda()
    X_val = torch.tensor(X[val_index], dtype=torch.float32).cuda()
    y_train = torch.tensor(y[trn_index, np.newaxis], dtype=torch.float32).cuda()
    y_val = torch.tensor(y[val_index, np.newaxis], dtype=torch.float32).cuda()
    
    print(f"Fold {i + 1}")
    model = LSTMNet(256, 512, X.shape, 30)
    model.cuda()
    
    valid_preds = train_model(
        model,
        X_train,
        y_train,
        X_val,
        y_val,
        f"best{i}.pt",
        anneal=False,
        epochs=50,
        seed=1029)
    
    train_preds[val_index] = valid_preds

Fold 1


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Epoch 1/50 loss=0.2602 val_loss=0.1995 val_mcc=0.2562 best_t=0.65 time=3.32s
Save Model on epoch 1
Epoch 2/50 loss=0.2090 val_loss=0.1813 val_mcc=0.3542 best_t=0.09 time=3.21s
Save Model on epoch 2
Epoch 3/50 loss=0.1833 val_loss=0.2863 val_mcc=0.6639 best_t=0.53 time=3.20s
Save Model on epoch 3
Epoch 4/50 loss=0.1740 val_loss=0.1267 val_mcc=0.6573 best_t=0.05 time=3.21s
Epoch 5/50 loss=0.1300 val_loss=0.1134 val_mcc=0.7389 best_t=0.08 time=3.22s
Save Model on epoch 5
Epoch 6/50 loss=0.1331 val_loss=0.1012 val_mcc=0.7335 best_t=0.32 time=3.22s
Epoch 7/50 loss=0.1179 val_loss=0.1185 val_mcc=0.7081 best_t=0.49 time=3.21s
Epoch 8/50 loss=0.1210 val_loss=0.0851 val_mcc=0.7630 best_t=0.09 time=3.22s
Save Model on epoch 8
Epoch 9/50 loss=0.1318 val_loss=0.0806 val_mcc=0.7655 best_t=0.24 time=3.25s
Save Model on epoch 9
Epoch 10/50 loss=0.1120 val_loss=0.1595 val_mcc=0.7655 best_t=0.66 time=3.24s
Epoch 11/50 loss=0.1296 val_loss=0.0903 val_mcc=0.7760 best_t=0.09 time=3.24s
Save Model on epoch

In [17]:
search_result = threshold_search(y, train_preds)
print("MCC: ", search_result["mcc"])
print("threshold: ", search_result["threshold"])

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


MCC:  0.6953482545840397
threshold:  0.31


In [18]:
meta_test = pd.read_csv('../input/metadata_test.csv')
meta_test = meta_test.set_index(['signal_id'])
meta_test.head()

,id_measurement,phase
signal_id,,
8712,2904,0
8713,2904,1
8714,2904,2
8715,2905,0
8716,2905,1


In [19]:
%%time
# First we daclarete a series of parameters to initiate the loading of the main data
# it is too large, it is impossible to load in one time, so we are doing it in dividing in 10 parts
first_sig = meta_test.index[0]
n_parts = 30
max_line = len(meta_test)
part_size = int(max_line / n_parts)
last_part = max_line % n_parts
print(first_sig, n_parts, max_line, part_size, last_part, n_parts * part_size + last_part)
# Here we create a list of lists with start index and end index for each of the 10 parts and one for the last partial part
start_end = [[x, x+part_size] for x in range(first_sig, max_line + first_sig, part_size)]
start_end = start_end[:-1] + [[start_end[-1][0], start_end[-1][0] + last_part]]
print(start_end)
X_test = []
# now, very like we did above with the train data, we convert the test data part by part
# transforming the 3 phases 800000 measurement in matrix (160,57)
for start, end in start_end:
    subset_test = pq.read_pandas('../input/test.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    for i in tqdm(subset_test.columns):
        id_measurement, phase = meta_test.loc[int(i)]
        subset_test_col = subset_test[i]
        subset_trans = transform_ts(subset_test_col, n_dim=n_dim)
        X_test.append([i, id_measurement, phase, subset_trans])

8712 30 20337 677 27 20337
[[8712, 9389], [9389, 10066], [10066, 10743], [10743, 11420], [11420, 12097], [12097, 12774], [12774, 13451], [13451, 14128], [14128, 14805], [14805, 15482], [15482, 16159], [16159, 16836], [16836, 17513], [17513, 18190], [18190, 18867], [18867, 19544], [19544, 20221], [20221, 20898], [20898, 21575], [21575, 22252], [22252, 22929], [22929, 23606], [23606, 24283], [24283, 24960], [24960, 25637], [25637, 26314], [26314, 26991], [26991, 27668], [27668, 28345], [28345, 29022], [29022, 29049]]


100%|██████████| 27/27 [00:02<00:00, 10.32it/s]

CPU times: user 35min 37s, sys: 29.8 s, total: 36min 6s
Wall time: 35min 16s


In [20]:
X_test_input = np.asarray([np.concatenate([X_test[i][3],X_test[i+1][3], X_test[i+2][3]], axis=1) for i in range(0,len(X_test), 3)])
np.save("X_test.npy",X_test_input)
X_test_input.shape

(6779, 160, 57)

In [21]:
submission = pd.read_csv('../input/sample_submission.csv')
print(len(submission))
submission.head()

20337


,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [22]:
test_batch_size = 300

X_test = torch.tensor(X_test_input, dtype=torch.float32).cuda()
test = torch.utils.data.TensorDataset(X_test)
test_loader = torch.utils.data.DataLoader(test, shuffle=False, batch_size=test_batch_size)

In [23]:
preds_test = []
for i in range(N_SPLITS):
    y_pred = np.zeros(X_test.size(0))
    model = LSTMNet(128, 64, X_test.shape)
    model.cuda()
    model.load_state_dict(torch.load(f"best{i}.pt"))
    model.eval()
    
    for j, (x_batch,) in enumerate(test_loader):
        y_p = model(x_batch).detach()
        y_pred[j * test_batch_size:(j+1) * test_batch_size] = sigmoid(y_p.cpu().numpy())[:, 0]
    pred_3 = []
    for pred_scalar in y_pred:
        for i in range(3):
            pred_3.append(pred_scalar)
    preds_test.append(pred_3)

TypeError: __init__() missing 1 required positional argument: 'n_attention'

In [24]:
preds_test = (np.squeeze(np.mean(preds_test, axis=0)) > search_result["threshold"]).astype(np.int)
preds_test.shape

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


()

In [25]:
submission["target"] = preds_test
submission.to_csv("submission.csv", index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0
